In [ ]:
#uses RegisteredMultiplier  - v1.0x
#uses MultiplyAdd - v1.1x

In [ ]:
import sys
import math
import os
import shutil

In [ ]:
def DotProduct_Systolic(parameters, Print_To_File, Files_Location, Is_Top, modules):
    
    N = parameters["N"] #N>=2

    #IN_WIDTH>=1
    IN_WIDTH = 10
    #INPUT_REG_DEPTH>=0
    INPUT_REG_DEPTH = 1
    #MULT_PIPE_DEPTH>=0
    MULT_PIPE_DEPTH = 1

    ModuleName="DotProduct_Systolic_{}".format(N)

    if Is_Top:
        Files_Location += ModuleName+"/"
        if not os.path.exists(Files_Location):
            os.makedirs(Files_Location)

    if Print_To_File<=0:
        of=sys.stdout
    else:
        of=open(Files_Location+ModuleName+".v", 'w+')

    print("`timescale 1ns / 1ps\n", file=of)
    print("module "+ModuleName, file=of)
    print("#(", file=of)
    print("parameter IN_WIDTH = {},".format(IN_WIDTH), file=of)
    print("parameter INPUT_REG_DEPTH = {},".format(INPUT_REG_DEPTH), file=of)
    print("parameter MULT_PIPE_DEPTH = {}".format(MULT_PIPE_DEPTH), file=of)
    print(")(", file=of)
    print("input clk, reset, enable,", file=of)
    print("input inReady,", file=of)
    print("input signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(N):
        print("A{}, ".format(i), end='', file=of)
    print(file=of)
    print("input signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(N):
        print("B{}, ".format(i), end='', file=of)
    print(file=of)
    print("output outReady,", file=of)
    lgn=math.ceil(math.log(N,2))
    al=-1+lgn
    if al==0:
        print("output signed [(2*IN_WIDTH):0] DP,", file=of)
    else:
        print("output signed [(2*IN_WIDTH)+{}:0] DP,".format(al), file=of)
    print("output earlyOutReady", file=of)
    print(");\n", file=of)

    print("wire signed [(2*IN_WIDTH)-1:0] M0;", file=of)
    print("RegisteredMultiplier #(.IN_WIDTH(IN_WIDTH),", file=of)
    print(".INPUT_REG_DEPTH(INPUT_REG_DEPTH),", file=of)
    print(".MULT_PIPE_DEPTH(MULT_PIPE_DEPTH))", file=of)
    print("Mult0(clk, reset, enable,", file=of)
    print("inReady,", file=of)
    print("A0, B0,", file=of)
    print("MOR0, //not used", file=of)
    print("M0,", file=of)
    print("eMOR0); //not used", file=of)
    print(file=of)

    i=0; twoPW=1; j=N-1; k=0;
    while(True):
        i+=1
        if i == twoPW:
            k+=1;
            if i==1:
                print("wire signed [2*IN_WIDTH:0] psl{} [0:{}];".format(k,twoPW-1), file=of)
            else:
                print("wire signed [(2*IN_WIDTH)+{}:0] psl{} [0:{}];".format(k-1,k,twoPW-1), file=of)
            twoPW = 2*twoPW
            j-=i
            if j==0:
                break
            elif j<twoPW:
                k+=1
                print("wire signed [(2*IN_WIDTH)+{}:0] psl{} [0:{}];".format(k-1,k,j-1), file=of)
                break
    print(file=of)

    i=0; twoPW=1; k=0; ok=0; l=0; ol=0;
    while(i<N-1):
        i+=1
        ol=l;
        l+=1;
        ok=k;
        if i == twoPW:
            k+=1;
            twoPW = 2*twoPW
            l=0;
        if ok==0:
            print("MultiplyAdd #( .IN_M_WIDTH(IN_WIDTH), .IN_A_WIDTH(2*IN_WIDTH), ", end='', file=of)
        else:
            print("MultiplyAdd #( .IN_M_WIDTH(IN_WIDTH), .IN_A_WIDTH((2*IN_WIDTH)+{}), ".format(ok), end='', file=of)
        print(".OUT_WIDTH((2*IN_WIDTH)+{}), ".format(k), end='', file=of)
        if i==1:
            print(".INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )", file=of)
        else:
            print(".INPUT_REG_DEPTH(INPUT_REG_DEPTH+{}), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )".format(i-1), file=of)
        print("MA{}(clk, reset, enable,".format(i), file=of)
        print("inReady,", file=of)
        if i ==1:
            print("A1, B1,\nM0,".format(i), file=of)
        else:
            print("A{}, B{},\npsl{}[{}],".format(i, i, ok,ol, i), file=of)
        if i==N-1:
            print("outReady,", file=of)
            print("psl{}[{}],".format(k,l), file=of)
            print("earlyOutReady);\n", file=of)
            print("assign DP = psl{}[{}];".format(k,l), file=of)
        else:
            print("OR{}, //not used".format(i), file=of)
            print("psl{}[{}],".format(k,l), file=of)
            print("eOR{}); //not used".format(i), file=of)
        print(file=of)

    print("endmodule", file=of)

    if Print_To_File>0:
        of.close()
    
    if "RegisteredMultiplier" not in modules:
        shutil.copy("./RegisteredMultiplier  - SRL_Input - v1.01.v",Files_Location+"RegisteredMultiplier.v") #no parameter
        modules["RegisteredMultiplier"]="RegisteredMultiplier"
    if "MultiplyAdd" not in modules:
        shutil.copy("./MultiplyAdd - SRL_Input - v1.11.v",Files_Location+"MultiplyAdd.v") #no parameter
        modules["MultiplyAdd"]="MultiplyAdd"

    modules[ModuleName]="DotProduct_Systolic"
    return ModuleName